In [1]:
import numpy as np
from scipy import linalg as lin
import nibabel as nib

In [2]:
pwd

'/home/jb/code/pasteur/brainspell-neo/mni-template'

In [3]:
mask_file = "MNI152_T1_3mm_brain_mask_scipy.nii"
mask_img = nib.load(mask_file)
header = mask_img.get_header()

In [4]:
print(header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [ 3 53 63 46  1  1  1  1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : label
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [-1.  3.  3.  3.  1.  0.  0.  0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 1.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'spm - 3D normalized'
aux_file        : b''
qform_code      : aligned
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 78.0
qoffset_y       : -112.0
qoffset_z       : -50.0
srow_x          : [ -3.   0.  

### dimension of the image

In [5]:
mask_img.shape

(53, 63, 46)

In [6]:
voxel_coo = np.asarray([0,0,0,1])
voxel_coo = voxel_coo[np.newaxis,:].T
print(voxel_coo)

[[0]
 [0]
 [0]
 [1]]


### Coordinates of the (0,0,0)

In [39]:
A = mask_img.get_affine()
talairach_coo = np.dot(A, voxel_coo)
print(talairach_coo)
print(A)

[[  72.]
 [-106.]
 [ -44.]
 [   1.]]
[[  -3.    0.    0.   78.]
 [   0.    3.    0. -112.]
 [   0.    0.    3.  -50.]
 [   0.    0.    0.    1.]]


### Voxel associated with these coordinates

In [8]:
talairach_coo = np.asarray([[-10, 20, 40, 1]]).T
invA = lin.inv(A)
#new_vox_coo = np.rint(invA.dot(talairach_coo))
new_vox_coo = (invA.dot(talairach_coo))
print(new_vox_coo)
print(np.rint(new_vox_coo))

[[ 29.33333333]
 [ 44.        ]
 [ 30.        ]
 [  1.        ]]
[[ 29.]
 [ 44.]
 [ 30.]
 [  1.]]


In [9]:
np.asarray(mask_img.shape).prod()*4

614376

### insert_at_location(self, value, x, y, z, width=1)

In [25]:
def construct_ball(dim=3, radius=1):
    """
    Create a ball in 3 dimension with radius 'radius' 
    Diameter will be 2*radius + 1
    
    parameters
    ----------
    dim: int
        dimension of the space - only 3 is implemented
    radius: int
        radius of the sphere
    """
    try:
        assert(dim==3)
    except:
        raise NotImplementedError
    
    # make a cube
    assert radius >= 1, print('radius >= 1')
    radius = np.rint(radius)
    cube_length = int(radius*2 + 1)
    ball = np.zeros((cube_length, cube_length, cube_length))

    # coord contains the range of coordinates
    coord = np.arange(cube_length)
    mesh = np.meshgrid(coord, coord, coord)
    
    # center the mesh coordinates
    mesh = tuple([mesh[i]-radius for i in range(dim)])
    
    # keep those less or equal to radius
    ball_coord = np.sqrt(mesh[0]**2 + mesh[1]**2 + mesh[2]**2) <= radius
    ball[np.where(ball_coord)] = 1
    
    return ball
    

In [29]:
ball2 = construct_ball(radius=2)
print(ball2)

[[[ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.]
  [ 0.  1.  1.  1.  0.]
  [ 0.  1.  1.  1.  0.]
  [ 0.  1.  1.  1.  0.]
  [ 0.  0.  0.  0.  0.]]

 [[ 0.  0.  1.  0.  0.]
  [ 0.  1.  1.  1.  0.]
  [ 1.  1.  1.  1.  1.]
  [ 0.  1.  1.  1.  0.]
  [ 0.  0.  1.  0.  0.]]

 [[ 0.  0.  0.  0.  0.]
  [ 0.  1.  1.  1.  0.]
  [ 0.  1.  1.  1.  0.]
  [ 0.  1.  1.  1.  0.]
  [ 0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]]


In [32]:
import scipy.ndimage.morphology as morph

In [38]:
brain = np.zeros(mask_img.shape, dtype='int32')

voxel_coo = np.asarray([2,2,2,1])
voxel_coo = voxel_coo[np.newaxis,:].T
talairach_coo = np.dot(A, voxel_coo)
new_vox_coo = np.rint(invA.dot(talairach_coo)).astype('int32')
assert(np.array_equal(new_vox_coo, voxel_coo))


brain[tuple(new_vox_coo[:3])] = 2
ball1 = construct_ball(radius=1)
brain = morph.binary_dilation(brain, structure=ball1)

brain[0:5,0:5,0:5]

array([[[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]],

       [[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False,  True, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]],

       [[False, False, False, False, False],
        [False, False,  True, False, False],
        [False,  True,  True,  True, False],
        [False, False,  True, False, False],
        [False, False, False, False, False]],

       [[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False,  True, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]],

       [[False, False, False, False, False],
        [False, False, False, False, False],
  

In [ ]:
def insert_at_location(brain_img, value, x, y, z, width=1):
    """
    Take a brain nifti image, add value around x,y,z coordinates
    
    """
    
    
    cube_length = width*2 + 1
    center = np.asarray([width, width, width])
    xdim = ydim = zdim = arange(cube_lenght)
    

In [45]:
voxel_coo = np.asarray([2,2,2,1])
voxel_coo.shape = (1,4)
voxel_coo

array([[2, 2, 2, 1]])

In [52]:
import scipy.stats as st
st.norm.sf([np.nan, 0, 3, -3])

/home/jb/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater
  cond1 = (scale > 0) & (x > self.a) & (x < self.b)
/home/jb/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in less
  cond1 = (scale > 0) & (x > self.a) & (x < self.b)
/home/jb/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1749: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


array([       nan,  0.5      ,  0.0013499,  0.9986501])

In [79]:
a = np.asarray([[0, 1, 2., 0, 0],[0, 1, 2., 0, 0]])
c = np.where(a != 0)
d = np.where(a==0)
a[d] = np.nan
print(a)
print(a == np.nan)
print(c)
a[c] = a[c]+.5
print(a)

[[ nan   1.   2.  nan  nan]
 [ nan   1.   2.  nan  nan]]
[[False False False False False]
 [False False False False False]]
(array([0, 0, 1, 1]), array([1, 2, 1, 2]))
[[ nan  1.5  2.5  nan  nan]
 [ nan  1.5  2.5  nan  nan]]


In [80]:
d = np.isnan(a)

In [81]:
d 

array([[ True, False, False,  True,  True],
       [ True, False, False,  True,  True]], dtype=bool)

In [84]:
m = ~np.isnan(a)
m

array([[False,  True,  True, False, False],
       [False,  True,  True, False, False]], dtype=bool)

In [83]:
c = a[~np.isnan(a)]
c

array([ 1.5,  2.5,  1.5,  2.5])

In [86]:
c.sort()

In [87]:
c


array([ 1.5,  1.5,  2.5,  2.5])